# Solving Poisson Equation in 1D via FEM

The goal of this notebook is to solve the boundary value problem (BVP) that has the form:

$$
\left\{
\begin{array}{ll}
    - \frac{d}{dx} \left( k(x) \frac{du(x)}{dx} \right)
    = f(x)  & \text{in} \; \Omega, \\
    u(x) = u_D & \text{on} \; \Gamma_D, \\
    k(x) \frac{du(x)}{dx} n = g & \text{on} \; \Gamma_N.
\end{array}
\right.
$$
Here, $\Gamma_D \subset \Omega$ denotes the part of the boundary where we prescribe Dirichlet boundary conditions, and $\Gamma_N \subset \Omega$ denotes the part of the boundary where we prescribe Neumann boundary conditions. $n$ is $-1$ at the left end and $1$ at the right end. 

__The weak form__ reads:

Find $u \in V_{u_D}$:
$$ \int_\Omega k \frac{du}{dx}\frac{dv}{dx} \, dx = \int_\Omega f\,v \, dx + \int_{\Gamma_N} g\,v\,ds, \quad \forall v \in V_0 $$
where
$$
\begin{align}
V_{u_D} &:= \left\{ v \in H^1(\Omega) \, |\, v = u_D \text{ on } \Gamma_D \right\},\\
V_{0} &:= \left\{ v \in H^1(\Omega) \, |\, v = 0 \text{ on } \Gamma_D \right\}.
\end{align}
$$

To obtain the finite element discretization we then introduce a mesh $\mathcal{T}_h$ of the domain $\Omega$ and we define a finite dimensional subspace $V_h \subset H^1(\Omega)$ consisting of globally continuous functions that are piecewise polynomial on each element of $\mathcal{T}_h$.

By letting $V_{h, u_D} := \{ v_h \in V_h \,|\, v_h = u_D \text{ on } \Gamma_D\}$ and $V_{h, 0} := \{ v_h \in V_h \,|\, v_h = 0 \text{ on } \Gamma_D\}$, the finite element method then reads:

Find $u_h \in V_{h, u_D}$ such that:
$$ \int_\Omega k\frac{d u_h}{dx} \frac{d v_h}{dx} \, dx = \int_\Omega f\,v_h \, dx + \int_{\Gamma_N} g\,v_h\,ds, \quad \forall v_h \in V_{h,0}. $$

In this example, $\Omega = [0,1], \Gamma_D = \{x | x = 0 \; \text{or} \; x = 1\}$, and $\Gamma_N = \emptyset$.
We also assume that $k(x) = 1$, $f(x) = 1$, and the boundary conditions are zero Dirichlet (i.e. $u_D = 0$).
The exact solution is $u(x) = 0.5x(1-x)$.

Last update: Oct. 10, 2021

Author: Noemi Petra (npetra@ucmerced.edu), modified by Ki-Tae Kim (kkim107@ucmerced.edu)

## Import packages

We import the following Python packages:

- `dolfin` is the python interface to FEniCS (the computational backend of FEniCS).
- `matplotlib` is a plotting library that produces figure similar to the Matlab ones.
- `numpy` is the python fundamental package for scientific computing (multidimensional arrays, linear algebra, etc.).

In [ ]:
import dolfin as dl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

## Create mesh and the finite element space

We define a mesh of the unit interval $\Omega = [0,1]$ with `n` elements. The mesh size $h$ is $\frac{1}{n}$.

We also define the finite element space $V_h \in H^1(\Omega)$ as the space of globally continuos functions that are piecewise polinomial (of degree $d$) on the elements of the mesh.

In [ ]:
# number of elements
n = 4

# degree of finite element basis functions
# 1-represents piecewise linear basis functions
# 2-represents piecewise quadratic basis functions
d = 1

# Create mesh and define function space
mesh = dl.UnitIntervalMesh(n)

# Define the function space
Vh = dl.FunctionSpace(mesh, 'Lagrange', d)

# Define the exact solution
u_true = dl.Expression('0.5*x[0]*(1.0-x[0])', degree=2)

# Show the mesh
dl.plot(mesh)
plt.show()

## Define the Dirichlet boundary condition

We define the Dirichlet boundary condition $u(0) = u(1) = 0$.

In [ ]:
# set the boundary conditions
# ud - instance holding the ud values
# ud_boundary - is a function describing whether a point lies on the
# boundary where u is specified; this returns a boolean value:
# True if x is on the Dirichlet boundary and False otherwise
def ud_boundary(x, on_boundary):
    return on_boundary

# Define boundary conditions
ud = dl.Constant('0.0')
bc = dl.DirichletBC(Vh, ud, ud_boundary)

## Define the variational problem

We write the variational problem $a(u_h, v_h) = L(v_h)$. Here, the bilinear form $a$ and the linear form $L$ are defined as

- $a(u_h, v_h) := \int_\Omega k\frac{d u_h}{dx} \frac{d v_h}{dx} \, dx$
- $L(v_h) := \int_\Omega f v_h \, dx + \int_{\Gamma_N} g \, v_h \, dx$.

$u_h$ denotes the trial function and $v_h$ denotes the test function.

In [ ]:
# Define variational problem
uh = dl.TrialFunction(Vh)
vh = dl.TestFunction(Vh)
f = dl.Constant(1.0)
a = dl.inner(dl.grad(uh), dl.grad(vh))*dl.dx
L = f*vh*dl.dx

## Assemble and solve the linear system

We now assemble the finite element stiffness matrix $A$ and the right hand side vector $b$. Dirichlet boundary conditions are applied at the end of the finite element assembly procedure and before solving the resulting linear system of equations.

In [ ]:
# represents uh as a function in a finite element function space Vh
uh = dl.Function(Vh)

# assemble system
A, b = dl.assemble_system(a, L, bc)
if mesh.num_cells() < 8: # print for small meshes only
    print(A.array())      # be careful with the identation!
    print(b.get_local())
    
# solve the linear system
dl.solve(A, uh.vector(), b)

# alternatively you can also (assemble + solve)
# dl.solve(a == L, uh, bc)

In [ ]:
# Plot solution
dl.plot(uh)

xtemp = np.linspace(0,1,100)
uexact = 0.5*xtemp*(1-xtemp)
plt.plot(xtemp, uexact)

plt.legend(['fe solution', 'exact'])
plt.xlabel('x')
plt.ylabel('u(x)')
plt.show()

## Compute error norm

We then compute the $L^2(\Omega)$ norm of the difference between the exact solution and the finite element approximation:
$
\sqrt{\int_\Omega (u_{exact} - u_h)^2 dx}.
$

In [ ]:
# Compute error in L2 norm
error_L2 = dl.errornorm(u_true, uh, 'L2')

# Print errors
print('error_L2  =', error_L2)